In [62]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt



Read data






In [63]:
input_data=pd.read_csv("weather.csv")
#input_data.info()
#input_data.describe()

pre processing